### type 2 dembedding

### Ce code dynamique est conçu pour gérer la classification de tous les types d'erreurs spécifiés dans le dataset. L'utilisateur peut sélectionner le type d'erreur à analyser directement via une saisie dans le terminal, ce qui déclenche un pipeline de nettoyage et de traitement adapté à ce type d'erreur. Le pipeline applique des étapes de prétraitement personnalisées, génère des embeddings vectoriels à l'aide de bge-m3, calcule une mesure de similarité entre les phrases source et simplifiées, et effectue une classification supervisée en utilisant la régression logistique. Cette approche flexible et automatisée permet de traiter chaque type d'erreur de manière spécifique et d'assurer un apprentissage performant tout en s'adaptant aux besoins de l'utilisateur.

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from imblearn.over_sampling import SMOTE
from FlagEmbedding import BGEM3FlagModel
import spacy

data=pd.read_csv("fusionned_file.csv",sep=";")
data.drop(columns=['run_id', 'snt_id','No error' ,'Commentaire'], inplace=True)  

# Charger SpaCy pour le traitement du langage
nlp = spacy.load("en_core_web_lg")

# Fonction pour supprimer la ponctuation
def remove_punctuation(text):
    return "".join([char for char in text if char.isalnum() or char.isspace()])

# Fonction pour convertir en minuscules
def to_lowercase(text):
    return text.lower()

# Fonction pour supprimer les stopwords
def remove_stopwords(text):
    doc = nlp(text)
    return " ".join([token.text for token in doc if not token.is_stop])

# Fonction pour effectuer la lemmatisation
def lemmatize(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

# Pipeline configurable
def clean_text(text, config):
    if config.get("remove_punctuation", False):
        text = remove_punctuation(text)
    if config.get("to_lowercase", False):
        text = to_lowercase(text)
    if config.get("remove_stopwords", False):
        text = remove_stopwords(text)
    if config.get("lemmatize", False):
        text = lemmatize(text)
    return text

# Configuration du pipeline pour différents types d’erreurs
pipeline_configs = {
    "Random generation": {
        "remove_punctuation": True,     # Les phrases générées aléatoirement peuvent inclure beaucoup de ponctuation inutile.
        "to_lowercase": True,           # Uniformiser les cas pour éviter les variations inutiles.
        "remove_stopwords": True,       # Les stopwords n'apportent aucune information pertinente.
        "lemmatize": True,              # Simplifier les mots générés aléatoirement.
    },
    "Syntax error": {
        "remove_punctuation": False,    # Garder la ponctuation pour identifier les erreurs syntaxiques.
        "to_lowercase": True,           # Uniformiser les textes pour une analyse syntaxique cohérente.
        "remove_stopwords": False,      # Les stopwords peuvent être nécessaires pour détecter les erreurs syntaxiques.
        "lemmatize": False,             # Éviter de modifier les structures pour préserver l'intégrité syntaxique.
    },
    "Contradiction": {
        "remove_punctuation": True,     # Supprimer la ponctuation pour se concentrer sur les contradictions sémantiques.
        "to_lowercase": True,
        "remove_stopwords": True,       # Les stopwords n'ont pas d'impact sur les contradictions.
        "lemmatize": True,              # Faciliter la détection de contradictions.
    },
    "Simple punctuation / grammar errors": {
        "remove_punctuation": False,    # Garder la ponctuation pour détecter les erreurs de ponctuation.
        "to_lowercase": True,           # Uniformiser les cas.
        "remove_stopwords": False,      # Conserver les stopwords pour ne pas altérer les phrases.
        "lemmatize": False,             # Ne pas modifier les formes des mots.
    },
    "Redundancy": {
        "remove_punctuation": True,     # Supprimer les ponctuations inutiles pour analyser la redondance.
        "to_lowercase": True,           
        "remove_stopwords": False,      # Conserver les stopwords pour identifier des répétitions complètes.
        "lemmatize": False,             # Ne pas modifier les mots pour détecter les répétitions exactes.
    },
    "Format misalignement": {
        "remove_punctuation": False,    # Garder la ponctuation pour vérifier les erreurs de format.
        "to_lowercase": True,           
        "remove_stopwords": False,      # Les stopwords peuvent être utiles pour détecter des problèmes de formatage.
        "lemmatize": False,             
    },
    "Prompt misalignement": {
        "remove_punctuation": False,     # Supprimer les caractères inutiles pour vérifier la correspondance avec le prompt.
        "to_lowercase": True,
        "remove_stopwords": True,       # Nettoyer les phrases pour se concentrer sur la correspondance avec le prompt.
        "lemmatize": True,              
    },
    "Out-of-Scope Generation": {
        "remove_punctuation": False,    # Garder la ponctuation pour analyser les phrases hors contexte.
        "to_lowercase": True,           
        "remove_stopwords": False,      
        "lemmatize": True,              
    },
    "Topic shift": {
        "remove_punctuation": False,   
        "to_lowercase": True,
        "remove_stopwords": False,    
        "lemmatize": True,              
    },
    "Oversimplification of Logical Arguments": {
        "remove_punctuation": True,     # Supprimer la ponctuation pour clarifier les arguments.
        "to_lowercase": True,
        "remove_stopwords": False,       
        "lemmatize": False,              
    },
    "Overgeneralization": {
        "remove_punctuation": True,     # Supprimer la ponctuation pour simplifier les phrases trop généralisées.
        "to_lowercase": True,
        "remove_stopwords": False,       
        "lemmatize": False,              
    },
    "Loss of Informative Content": {
        "remove_punctuation": True,     # Supprimer les ponctuations inutiles pour clarifier les informations perdues.
        "to_lowercase": True,
        "remove_stopwords": False,       
        "lemmatize": False,              
    },
    "Factuality hallucination": {
        "remove_punctuation": True,     # Supprimer les ponctuations inutiles pour analyser les faits.
        "to_lowercase": True,
        "remove_stopwords": False,       
        "lemmatize": True,              
    },
    "Faithfulness hallucination": {
        "remove_punctuation": True,     # Supprimer la ponctuation pour se concentrer sur la fidélité.
        "to_lowercase": True,
        "remove_stopwords": False,       
        "lemmatize": True,              
    },
}

# Liste des colonnes d'erreurs
error_columns = [
    'Random generation', 'Syntax error', 'Contradiction',
    'Simple punctuation / grammar errors', 'Redundancy', 'Format misalignement',
    'Prompt misalignement', 'Out-of-Scope Generation', 'Topic shift',
    'Oversimplification of Logical Arguments', 'Overgeneralization',
    'Loss of Informative Content', 'Factuality hallucination', 'Faithfulness hallucination'
]

# Demander à l'utilisateur de choisir une colonne cible
print("Voici les types d'erreurs disponibles :")
for i, col in enumerate(error_columns):
    print(f"{i + 1}. {col}")

choice = int(input("Entrez le numéro correspondant à la cible souhaitée : ")) - 1
target_column = error_columns[choice]
print(f"Vous avez choisi : {target_column}")



# Définir la colonne cible
data['target'] = data[target_column].astype(int)

# Appliquer le pipeline de nettoyage
config = pipeline_configs.get(target_column, {})
data['cleaned_source'] = data['source sentence'].apply(lambda x: clean_text(x, config))
data['cleaned_simplified'] = data['simplified sentence'].apply(lambda x: clean_text(x, config))

# Charger le modèle BGEM3FlagModel
embedding_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

# Générer les embeddings
data['source_latent'] = data['cleaned_source'].apply(embedding_model.encode)
data['simplified_latent'] = data['cleaned_simplified'].apply(embedding_model.encode)

# Extraire les vecteurs denses
data['source_embedding'] = data['source_latent'].apply(lambda x: np.array(x['dense_vecs']))
data['simplified_embedding'] = data['simplified_latent'].apply(lambda x: np.array(x['dense_vecs']))

# Supprimer les colonnes intermédiaires
data = data.drop(columns=['source_latent', 'simplified_latent'])

# Calculer la similarité cosinus
data['similarity'] = data.apply(
    lambda row: cosine_similarity([row['source_embedding']], [row['simplified_embedding']])[0][0], axis=1
)

# Combiner les embeddings et la similarité
X = np.array([
    np.concatenate([row['source_embedding'], row['simplified_embedding'], [row['similarity']]])
    for _, row in data.iterrows()
])
y = data['target'].values

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Entraîner et évaluer Logistic Regression
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train_resampled, y_train_resampled)
y_pred = logistic_model.predict(X_test)

print("\nConfusion Matrix de Logistic Regression:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report de Logistic Regression:")
print(classification_report(y_test, y_pred, target_names=["False", "True"]))
print(f"Balanced Accuracy de Logistic Regression: {balanced_accuracy_score(y_test, y_pred):.3f}")

# Entraîner et évaluer SVM
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_resampled, y_train_resampled)
y_pred_svm = svm_model.predict(X_test)

print("\nConfusion Matrix de SVM:")
print(confusion_matrix(y_test, y_pred_svm))
print("\nClassification Report de SVM:")
print(classification_report(y_test, y_pred_svm, target_names=["False", "True"]))
print(f"Balanced Accuracy de SVM: {balanced_accuracy_score(y_test, y_pred_svm):.3f}")


Voici les types d'erreurs disponibles :
1. Random generation
2. Syntax error
3. Contradiction
4. Simple punctuation / grammar errors
5. Redundancy
6. Format misalignement
7. Prompt misalignement
8. Out-of-Scope Generation
9. Topic shift
10. Oversimplification of Logical Arguments
11. Overgeneralization
12. Loss of Informative Content
13. Factuality hallucination
14. Faithfulness hallucination
Vous avez choisi : Out-of-Scope Generation


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



Confusion Matrix de Logistic Regression:
[[48  8]
 [ 0 10]]

Classification Report de Logistic Regression:
              precision    recall  f1-score   support

       False       1.00      0.86      0.92        56
        True       0.56      1.00      0.71        10

    accuracy                           0.88        66
   macro avg       0.78      0.93      0.82        66
weighted avg       0.93      0.88      0.89        66

Balanced Accuracy de Logistic Regression: 0.929

Confusion Matrix de SVM:
[[51  5]
 [ 3  7]]

Classification Report de SVM:
              precision    recall  f1-score   support

       False       0.94      0.91      0.93        56
        True       0.58      0.70      0.64        10

    accuracy                           0.88        66
   macro avg       0.76      0.81      0.78        66
weighted avg       0.89      0.88      0.88        66

Balanced Accuracy de SVM: 0.805


In [16]:
# Fonction pour prédire si une erreur est présente ou non
def predict_error(user_source, user_simplified, logistic_model, config):
    # Nettoyer les phrases avec le pipeline configuré
    cleaned_source = clean_text(user_source, config)
    cleaned_simplified = clean_text(user_simplified, config)

    # Générer les embeddings avec le modèle d'embedding
    source_embedding = embedding_model.encode(cleaned_source)
    simplified_embedding = embedding_model.encode(cleaned_simplified)

    # Si les embeddings retournent un dictionnaire, extraire les vecteurs denses
    if isinstance(source_embedding, dict) and 'dense_vecs' in source_embedding:
        source_embedding = np.array(source_embedding['dense_vecs'])
    if isinstance(simplified_embedding, dict) and 'dense_vecs' in simplified_embedding:
        simplified_embedding = np.array(simplified_embedding['dense_vecs'])

    # Calculer la similarité cosinus
    similarity = cosine_similarity([source_embedding], [simplified_embedding])[0][0]

    # Combiner les embeddings et la similarité en un seul vecteur
    combined_features = np.concatenate([source_embedding, simplified_embedding, [similarity]])

    # Prédire à l'aide du modèle entraîné
    prediction = logistic_model.predict([combined_features])
    return prediction[0]

# Demander à l'utilisateur de saisir des phrases
user_source = input("Entrez la phrase source : ")
user_simplified = input("Entrez la phrase simplifiée : ")

# Prédire le type d'erreur
user_prediction = predict_error(user_source, user_simplified, logistic_model, config)

# Afficher le résultat
if user_prediction == 1:
    print(f"L'erreur '{target_column}' est présente dans cette simplification.")
else:
    print(f"L'erreur '{target_column}' n'est pas présente dans cette simplification.")



L'erreur 'Out-of-Scope Generation' est présente dans cette simplification.
